# Добро пожаловать в практическую часть


Вы находитесь в онлайн-сервисе Google Colab. Этот сервис используется для написания кода и предоставляет мощности для обучения моделей.

В рамках практической части мы пройдемся по основным этапам обучения модели. Весь код уже написан и логически разбит по ячейкам. Чтобы все отработало неоходимо последовательно запускать ячейки. Для запуска ячейки необходимо нажать круглую кнопку с треугольником в левом верхнем углу запускаемой ячейки.


---

### Нулевой шаг

Перед началом работы переключимся на использование мощностей видеокарты. Это позволит обучать модель значительно быстрее.

Для этого нужно нажать вверху "Изменить", затем "Настройки блокнота" и убедиться, что под словами "Аппартаный ускоритель" написано "GPU"

### Установка модулей

In [9]:
# Первым делом нам нужно установить дополнительные модули для Python.
# pyTelegramBotAPI необходим для работы с телеграмботом
!pip install pyTelegramBotAPI
# transformers необходим для работы с моделями
!pip install transformers

# Для запуска ячейки необходимо нажать круглую кнопку с треугольником в левом верхнем углу запускаемой ячейки

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=9f48eb9fe02dd205413c3910dd4bd831ce3e2a4ff68c3015163d25c33b73c58d
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00


## Активация необходимых модулей и доступа к гугл-диску

In [10]:
# Какие-то модули уже стоят в питоне изначально, какие-то мы поставили выше. Сейчас их необходимо автивировать

import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup,AutoTokenizer,AutoModelForCausalLM
from tqdm import tqdm, trange
import torch.nn.functional as F
from re import *

In [11]:
# Разрешаем нашему коду смотреть в гугл-диск
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Исходные данные

In [13]:
# Открываем файл с исходными данными
with open('/content/drive/MyDrive/Практическое_занятие/данные/esse.txt','r', encoding='utf-8') as f:
  opened_texts = f.read()

data = findall('<s>(Тема: .*?)\n', opened_texts) #Найдем все темы эссе
data[:10]

['Тема: «Общество – свод камней, который обрушился бы, если бы один не поддержал другого» (Сенека)',
 'Тема: Как раскрывается И.А. Буниным философская тема в рассказе «Господин из Сан-Франциско»?',
 'Тема: Закон тщетно существует для тех, кто не имеет мужества и средств его защищать».',
 'Тема: Бизнес-это искусство извлекать деньги из кармана другого человека, не прибегая к насилию.',
 'Тема: Соглашения предотвращают конфликты',
 'Тема: Сказать, что человек состоит из силы и слабости, из разумения иослепления, из ничтожества и величия, — это значит не осудить его, аопределить его сущность». Дидро Д.',
 'Тема: «Личность человека, ни в каком смысле не является предсуществующей по отношению к его деятельности, как и его сознание, она ею порождается» (А.Н. Леонтьев) «Личность – это человек как носитель сознания» (К.К. Платонов)',
 'Тема: Историческое сочинение 1157–1174 года',
 'Тема: «Поэт бесконечно уступает художнику в изображении видимого, и музыканту -в изображении невидимого».(Леонар

In [14]:
# Добавим к каждому тексту тег окончания текста, чтобы модель понимала где обычно они заканчиваются
texts = '<|endoftext|>'.join([text for text in data])

In [15]:
# Проверим, как выглядит текст после изменений. Возьмем первые 1000 символов
texts[:1000]

'Тема: «Общество – свод камней, который обрушился бы, если бы один не поддержал другого» (Сенека)<|endoftext|>Тема: Как раскрывается И.А. Буниным философская тема в рассказе «Господин из Сан-Франциско»?<|endoftext|>Тема: Закон тщетно существует для тех, кто не имеет мужества и средств его защищать».<|endoftext|>Тема: Бизнес-это искусство извлекать деньги из кармана другого человека, не прибегая к насилию.<|endoftext|>Тема: Соглашения предотвращают конфликты<|endoftext|>Тема: Сказать, что человек состоит из силы и слабости, из разумения иослепления, из ничтожества и величия, — это значит не осудить его, аопределить его сущность». Дидро Д.<|endoftext|>Тема: «Личность человека, ни в каком смысле не является предсуществующей по отношению к его деятельности, как и его сознание, она ею порождается» (А.Н. Леонтьев) «Личность – это человек как носитель сознания» (К.К. Платонов)<|endoftext|>Тема: Историческое сочинение 1157–1174 года<|endoftext|>Тема: «Поэт бесконечно уступает художнику в изобр

In [16]:
# Сохраним наши новые данные в файл
with open('/content/drive/MyDrive/Практическое_занятие/данные/e_text.txt','w', encoding='utf-8') as f:
  f.write(texts)

In [17]:
# Посмотрим, что они корректно сохранились
with open('/content/drive/MyDrive/Практическое_занятие/данные/e_text.txt','r', encoding='utf-8') as f:
  opened_texts = f.read()

opened_texts[:1000]

'Тема: «Общество – свод камней, который обрушился бы, если бы один не поддержал другого» (Сенека)<|endoftext|>Тема: Как раскрывается И.А. Буниным философская тема в рассказе «Господин из Сан-Франциско»?<|endoftext|>Тема: Закон тщетно существует для тех, кто не имеет мужества и средств его защищать».<|endoftext|>Тема: Бизнес-это искусство извлекать деньги из кармана другого человека, не прибегая к насилию.<|endoftext|>Тема: Соглашения предотвращают конфликты<|endoftext|>Тема: Сказать, что человек состоит из силы и слабости, из разумения иослепления, из ничтожества и величия, — это значит не осудить его, аопределить его сущность». Дидро Д.<|endoftext|>Тема: «Личность человека, ни в каком смысле не является предсуществующей по отношению к его деятельности, как и его сознание, она ею порождается» (А.Н. Леонтьев) «Личность – это человек как носитель сознания» (К.К. Платонов)<|endoftext|>Тема: Историческое сочинение 1157–1174 года<|endoftext|>Тема: «Поэт бесконечно уступает художнику в изобр

### Запускаем обучение модели

In [ ]:
%%time
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/Практическое_занятие/ru-gpts/pretrain_transformers.py \
    --output_dir=our_finetune_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=/content/drive/MyDrive/Практическое_занятие/данные/e_text.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 250 \
    --block_size 1024 \
    --overwrite_output_dir

2023-06-24 07:55:23.060942: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
06/24/2023 07:55:24 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
06/24/2023 07:55:50 - INFO - __main__ -   Training/evaluation parameters Namespace(train_data_file='/content/drive/MyDrive/Практическое_занятие/данные/e_text.txt', output_dir='our_finetune_model', model_type='gpt2', eval_data_file=None, line_b

## Генерируем текст

In [ ]:
model_name_or_path = "/content/our_finetune_model" # Указываем путь до папки с нашей моделью
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path) # Активируем токенизатор - инуструмент, который превратит наш текст в нужный для модели формат
gpt = AutoModelForCausalLM.from_pretrained(model_name_or_path) # Активируем модель

### Запускаем генератор текста

Значение prompt отвечает за начало генерируемого текста. Его можно изменить, для лучшего результата сохранив тематику данных.

Параметры, которые можно позименять, чтобы изменить результат генерации:

*   max_length - максимальная длинна генерируемого текста в символах

*   num_return_sequences - сколько текстов сгенерирует модель. Чем больше текстов, тем дольше модель занимается генерацией

*   temperature - Чем выше этот параметр, тем более разнообразные тексты получаются. Однако, с повышением этого параметра повышается и вероятность, что текст получится менее похожим на человеческий

In [ ]:
prompt = 'Тема: Творчество кота' # Начальные слова, которые подаются модели. С них и будет начинаться генерируемый текст

input_ids = tokenizer.encode(prompt, return_tensors="pt") # С помощью токенизатора преобразуем начальные слова в формат для модели

# Функция генерации текста
# Параметры, которые можно позименять, чтобы изменить результат генерации:
#   max_length - максимальная длинна генерируемого текста в символах
#   num_return_sequences - сколько текстов сгенерирует модель. Чем больше текстов, тем дольше модель занимается генерацией
#   temperature - Чем выше этот параметр, тем более разнообразные тексты получаются.
#                 Однако, с повышением этого параметра повышается и вероятность, что текст получится менее похожим на человеческий
generated_text_samples = gpt.generate(
    input_ids,
    max_length=100,
    num_return_sequences=5,
    no_repeat_ngram_size=3,
    repetition_penalty=1.5,
    top_p=1.,
    temperature=1,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

# Выводим результат генерации
for i, beam in enumerate(generated_text_samples):
  print('')
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=False).split('<|endoftext|>')[0]))
  print()

### Создаем ТГ Бота

Откройте мессенджер Telegram, войдите в вашу учетную запись.


1.   Введите в поле поиска @BotFather и выберите бота.

2.   Нажмите «Запустить» для активации бота BotFather. В ответ вы получите список команд по управлению ботов.

3.   Выберите или напечатайте и отправьте команду /newbot.

4.   Дайте имя боту — пользователи увидят это имя при общении с ботом. И никнейм бота — по нему можно будет найти бота в Telegram. Никнейм должен быть уникальным, не повторять существующие в базе и заканчиваться на слово «bot».

После того как вы выберите подходящее имя бот будет создан. Вы получите сообщение со ссылкой на бота t.me/<никнейм_бота>

Для подключения модели к боту нам понадобится токен


In [ ]:
import telebot
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer


def model(text, mo, max_length=42):
  input_ids = tokenizer.encode(text, return_tensors="pt")

  generated_text_samples = gpt.generate(
      input_ids,
      max_length=100,
      num_return_sequences=1,
      no_repeat_ngram_size=3,
      repetition_penalty=1.5,
      top_p=1.,
      temperature=1,
      do_sample=True,
      top_k=125,
      early_stopping=True
  )
  return tokenizer.decode(generated_text_samples[0], skip_special_tokens=False).split('<|endoftext|>')[0]

bot = telebot.TeleBot('') # В этой трочке необходимо вставить токен от Вашего бота

@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Пожалуйста, напишите как Вы хотите начать свой рекламнй текст')

@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, model(message.text, gpt))
    bot.send_message(message.chat.id, 'Если хотите продолжить, напишите, пожалуйста, начало следующего рекламного текста')

bot.polling(none_stop=True, interval=0)